In [1]:
!pip3 install pandas
!pip3 install tensorflow
import pandas as pd
import numpy as np
import tensorflow as tf

# Assuming you have a dataset like the one you provided
# "Date" column should be in datetime format
# "Receipt_Count" column represents the number of receipts received on each day

# Load your dataset


df = pd.read_csv('data_daily.csv')
print(df.head(5))
print(df.columns)
print(df.dtypes)
df['# Date'] = pd.to_datetime(df['# Date'])
df['Month'] = df['# Date'].dt.month
df['Year'] = df['# Date'].dt.year

# Feature engineering
df_agg = df.groupby(['Year', 'Month'], as_index=False)['Receipt_Count'].sum()
df_agg['Prev_Month_Receipts'] = df_agg['Receipt_Count'].shift(1)

# Drop the first row as it will have NaN due to shift
df_agg = df_agg.dropna()

# Model data preparation
X = df_agg[['Prev_Month_Receipts']].values
y = df_agg['Receipt_Count'].values

# Normalize data manually
X_mean, X_std = X.mean(), X.std()
X_normalized = (X - X_mean) / X_std

y_mean, y_std = y.mean(), y.std()
y_normalized = (y - y_mean) / y_std

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(len(X_normalized) * split_ratio)

X_train, X_test = X_normalized[:split_index], X_normalized[split_index:]
y_train, y_test = y_normalized[:split_index], y_normalized[split_index:]

# Model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=1, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer, loss='mean_squared_error')

# Training
model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=2)

# Evaluation
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

# Inference for the year 2022
# Assuming new_data is the input for each day's receipts in 2021
# predicted_normalized = model.predict(X_normalized).flatten()
# predicted = predicted_normalized * y_std + y_mean

# # Print the predicted number of receipts for each month of 2022
# print(predicted)


# Generate predictions for the next 12 values
future_values = []

# Use the last sequence of known values to predict the next value

last_sequence = X_normalized[-1]
for _ in range(12):
    prediction = model.predict(last_sequence)
    future_values.append(prediction.flatten())
    last_sequence = prediction.flatten()
   

# Denormalize the predictions
future_values = np.array(future_values) * y_std + y_mean

print(future_values)



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


       # Date  Receipt_Count
0  2021-01-01        7564766
1  2021-01-02        7455524
2  2021-01-03        7095414
3  2021-01-04        7666163
4  2021-01-05        7771289
Index(['# Date', 'Receipt_Count'], dtype='object')
# Date           object
Receipt_Count     int64
dtype: object
Epoch 1/100
1/1 - 0s - loss: 1.3853 - 211ms/epoch - 211ms/step
Epoch 2/100
1/1 - 0s - loss: 1.1248 - 5ms/epoch - 5ms/step
Epoch 3/100
1/1 - 0s - loss: 0.9086 - 5ms/epoch - 5ms/step
Epoch 4/100
1/1 - 0s - loss: 0.7326 - 3ms/epoch - 3ms/step
Epoch 5/100
1/1 - 0s - loss: 0.5937 - 2ms/epoch - 2ms/step
Epoch 6/100
1/1 - 0s - loss: 0.4853 - 2ms/epoch - 2ms/step
Epoch 7/100
1/1 - 0s - loss: 0.3998 - 2ms/epoch - 2ms/step
Epoch 8/100
1/1 - 0s - loss: 0.3344 - 1ms/epoch - 1ms/step
Epoch 9/100
1/1 - 0s - loss: 0.2869 - 2ms/epoch - 2ms/step
Epoch 10/100
1/1 - 0s - loss: 0.2547 - 2ms/epoch - 2ms/step
Epoch 11/100
1/1 - 0s - loss: 0.2363 - 2ms/epoch - 2ms/step
Epoch 12/100
1/1 - 0s - loss: 0.2297 - 2ms/epoch - 2ms/ste

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf


# Provided data
df = pd.read_csv('data_daily.csv')
print(df.head(5))
print(df.columns)
print(df.dtypes)
df['# Date'] = pd.to_datetime(df['# Date'])
df['Month'] = df['# Date'].dt.month
df['Year'] = df['# Date'].dt.year

# Feature engineering
df_agg = df.groupby(['Year', 'Month'], as_index=False)['Receipt_Count'].sum()


# Take the difference between consecutive values
df_agg['Diff_Values'] = df_agg['Receipt_Count'].diff().fillna(0)

# Create input and target data for training
X_train = df_agg['Diff_Values'][:-1].values.reshape(-1, 1, 1)
y_train = df_agg['Diff_Values'][1:].values.reshape(-1, 1)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=1, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer, loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=2)

# Generate predictions for the next 12 values
X_pred = df_agg['Diff_Values'].values[-1].reshape(1, 1, 1)
y_pred_diff = []
for _ in range(12):
    pred_diff = model.predict(X_pred)
    y_pred_diff.append(pred_diff.flatten())
    X_pred = pred_diff.reshape(1, 1, 1)

print("1:",df_agg['Diff_Values'].values)
print("2:", y_pred_diff)
res = np.concatenate(y_pred_diff)
print("3:", res)
# Cumulatively sum the differences to obtain predictions for the original time series
y_pred_cumsum = np.cumsum(np.concatenate([df_agg['Diff_Values'].values, res]))
                                          
# print(y_pred_cumsum)
                                          
predictions = []
dec2021_val = 309948684
i = 1
j = 1
fir = 309948684 + res[0]
predictions.append(fir)
while(i<12 and j<12):
    predictions.append(predictions[i-1] + y_pred_cumsum[j])
    i+=1
    j+=1
print(predictions)
    
                                 

       # Date  Receipt_Count
0  2021-01-01        7564766
1  2021-01-02        7455524
2  2021-01-03        7095414
3  2021-01-04        7666163
4  2021-01-05        7771289
Index(['# Date', 'Receipt_Count'], dtype='object')
# Date           object
Receipt_Count     int64
dtype: object
Epoch 1/100
2/2 - 0s - loss: 204628582989824.0000 - 136ms/epoch - 68ms/step
Epoch 2/100
2/2 - 0s - loss: 180781733380096.0000 - 2ms/epoch - 831us/step
Epoch 3/100
2/2 - 0s - loss: 161792122683392.0000 - 2ms/epoch - 778us/step
Epoch 4/100
2/2 - 0s - loss: 153325651623936.0000 - 1ms/epoch - 692us/step
Epoch 5/100
2/2 - 0s - loss: 149749202157568.0000 - 1ms/epoch - 697us/step
Epoch 6/100
2/2 - 0s - loss: 141818645708800.0000 - 1ms/epoch - 664us/step
Epoch 7/100
2/2 - 0s - loss: 130327703977984.0000 - 2ms/epoch - 1ms/step
Epoch 8/100
2/2 - 0s - loss: 125457907318784.0000 - 2ms/epoch - 936us/step
Epoch 9/100
2/2 - 0s - loss: 118509740752896.0000 - 2ms/epoch - 1ms/step
Epoch 10/100
2/2 - 0s - loss: 11193125175

1/1 [==============================] - 0s 8ms/step
1: [        0. -16703227.  28575165.   2036205.  12506918.  -2494908.
  14119163.   9167228.  -2797077.  14819031.    119977.  13863522.]
2: [array([1547677.8], dtype=float32), array([172780.88], dtype=float32), array([19292.275], dtype=float32), array([2157.3562], dtype=float32), array([244.47449], dtype=float32), array([30.927177], dtype=float32), array([7.0874114], dtype=float32), array([3.900337], dtype=float32), array([2.9250326], dtype=float32), array([2.6003547], dtype=float32), array([2.4922698], dtype=float32), array([2.456288], dtype=float32)]
3: [1.5476778e+06 1.7278088e+05 1.9292275e+04 2.1573562e+03 2.4447449e+02
 3.0927177e+01 7.0874114e+00 3.9003370e+00 2.9250326e+00 2.6003547e+00
 2.4922698e+00 2.4562881e+00]
[311496361.75, 294793134.75, 306665072.75, 320573215.75, 346988276.75, 370908429.75, 408947745.75, 456154289.75, 500563756.75, 559792254.75, 619140729.75, 692352726.75]


In [4]:
a = 0.7
b = 0.3
final = []
for i in range(12):
    final.append(a * future_values[i][0] + b * predictions[i])
print(final)

[291593394.92499995, 285369444.425, 288409553.825, 292357795.125, 300149414.22499996, 307201140.125, 318483350.92499995, 332510242.125, 345692320.625, 363314127.625, 380966462.125, 402772029.225]
